In [1]:
import os
import numpy as np
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import random
import pickle

#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)

In [2]:
file_path = './data/preprocessed_user_item_rating.csv'

In [3]:
if not (os.path.exists(file_path)):
  raise ValueError('El archivo preprocessed_user_item_rating.csv no fue encontrado en el path')
else:
  print("El archivo ha sido cargado")

El archivo ha sido cargado


In [4]:
ratings=pd.read_csv(file_path, sep = ',', header=0, names = [ 'userid', 'artist-name', 'rating' ] )
ratings = ratings.loc[:,['userid', 'artist-name','rating']]
ratings

,userid,artist-name,rating
0,user_001000,Wilco,5.0
1,user_001000,Radiohead,4.9
2,user_001000,Animal Collective,4.7
3,user_001000,Girl Talk,4.6
4,user_001000,Aesop Rock,4.2
...,...,...,...
896880,user_000001,Jamie Lidell,0.2
896881,user_000001,Nick Holmes,0.2
896882,user_000001,Nuyorican Soul,0.2
896883,user_000001,The Birthday,0.2


In [5]:
ratings.rating.describe()

count    167494.000000
mean          0.635761
std           0.737520
min           0.200000
25%           0.200000
50%           0.400000
75%           0.700000
max           5.000000
Name: rating, dtype: float64

In [6]:
# Creacion sistema de recomendacion

In [7]:
reader = Reader( rating_scale = ( 0, 5 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ [ 'userid', 'artist-name', 'rating' ] ], reader )

In [8]:
trainset, testset=  train_test_split(surprise_dataset, test_size=.2)

### Modelo basado en distancias coseno

In [9]:
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud item-item
               }
algo = KNNBasic(k=30, min_k=5, sim_options=sim_options)

In [10]:
predictions = algo.fit(trainset).test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [11]:
pickle.dump( predictions, open( "./Data/predictions_uu_cosine.p", "wb" ) )

### Modelo basado en índice de Jaccard

### Modelo basado en correlación de Pearson

In [12]:
sim_options = {'name': 'pearson_baseline',
               'user_based': True,
               'shrinkage': 0  # no shrinkage
               }
algo = KNNBasic(sim_options=sim_options)

In [13]:
predictions = algo.fit(trainset).test(testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [14]:
pickle.dump( predictions, open( "./Data/predictions_uu_pearson.p", "wb" ) )

### Recomendaciones

In [15]:
pickle_predictions = pickle.load( open( "./Data/predictions_uu_cosine.p", "rb" ) )

In [16]:
#Predicciones para usuario user_001000
user_predictions=list(filter(lambda x: x[0]=='user_001000' and x[4]['was_impossible']==False,predictions))

In [17]:
#Ordenamos de mayor a menor estimación de relevancia
user_predictions.sort(key=lambda x : x.est, reverse=True)

In [18]:
#tomamos las 10 primeras predicciones
user_predictions=user_predictions[0:10]

In [19]:
#Se convierte a dataframe
labels = ['artist', 'estimation']
df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_predictions)), columns=labels)

### Recomendaciones usuario 001000

In [20]:
df_predictions

,artist,estimation
0,El Ten Eleven,2.800000
1,Bon Iver,1.205836
2,Sigur Rós,1.179212
3,Jack Johnson,1.171126
4,Bloc Party,1.169871
5,Ben Folds,1.007055
6,Crystal Castles,0.998564
7,Modest Mouse,0.949810
8,Rilo Kiley,0.849064
9,Massive Attack,0.835158


In [21]:
accuracy.rmse( user_predictions, verbose = True )

RMSE: 0.8964


0.8964139610329852